In [12]:
import os
import tweepy as tw
import pandas as pd

In [13]:
consumer_key= 'lL5jwhZjazxKKy4WUJkcRHsLb'
consumer_secret= 'r2f5XZ8f2bNtFSY72QZfdtrsNglGzIeRaHva2LoyZpVs14i0i8'
access_token= '995760376808591361-Lkl0hmd0zsjCopgGZgf4sbWppCWOYqb'
access_token_secret= 'ZMk77Ahi4s9IEfiCiK7riU7BeR6D7LG4C8o2We4pJpVa4'


In [16]:
#Module to fetch the last 200 tweets of a suspected user


auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)


prefixes=('@','#','http')
retweet=('RT')
def get_tweets(username): 
          
        + +
        
        # 200 tweets to b+
        e extracted 
        number_of_tweets=300
        tweets = api.user_timeline(screen_name=username, q="#* -filter:retweets") 
  
        # Empty Array 
        tmp=[] 
  
        # create array of tweet information: username,  
        # tweet id, date/time, text 
        tweets_for_csv = [tweet.text for tweet in tweets] # CSV file created  
        for j in tweets_for_csv: 
            #print (j,"\n")
            #clean text, split into words and rmeove hyperlinks, hashtags and @
            x=j.split()
            #print(x)
            for word in x[:]:
                if word.startswith(prefixes):
                    x.remove(word)
            
            #print(x)
            j=' '.join(word for word in x)
  
            # Appending tweets to the empty array tmp 
            if(j!=''):
                tmp.append(j)  
  
        # Printing the tweets 
        return tmp
  

In [17]:
df=pd.read_csv("CSV_files/Crime_Flagged.csv")
df

,Unnamed: 0,User,Location,Tweet,Flag,1,2,3,4
0,0,atxleb,NaN,Because they’d be prosecuting government offic...,0.0,analytical,NaN,NaN,NaN
1,1,ShayneOfficial,NaN,It’s actually cheaper to eat on the right. Tha...,0.0,joy,NaN,NaN,NaN
2,2,JamesJgm26,NaN,I will kill trump just to piss off the Russians,1.0,anger,tentative,NaN,NaN
3,3,VerucaMarie,NaN,always. We had to put the rock to sleep early ...,1.0,analytical,confident,NaN,NaN
4,4,Her_Masterpiece,"New York, USA",Just passed by a house with their icicle light...,0.0,tentative,NaN,NaN,NaN


In [18]:
import json
from ibm_watson import PersonalityInsightsV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException
from ibm_watson import PersonalityInsightsV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from os.path import join, dirname
import json
        

for i in df.itertuples():
    flag=i[5]
    if(flag==1):
        user=i[2]
        print("Suspected:",user)
        #Get all tweets from this user
        #Generate ego network for the user
        # Here goes the twitter handle for the user whose tweets are to be extracted. 
        cleanedtweet=get_tweets(user)  
        d={}#The dictionary
        dlist=[]#The list to append the dictionary to

        for j in cleanedtweet:
            d={}
            d['content']=j
            d['contenttype']="text/plain"
            d['created']= 0
            d['id']=i[1]
            d['language']= "en"
            dlist.append(d)
            #print(d,"\n")
        #print(dlist,"\n")
        with open('profile_demo.json', 'w') as outfile:
            json.dump({'contentItems': dlist}, outfile,indent=4)
        
        #Connect with Watson Personlity Insight
        
        authenticator = IAMAuthenticator('4_vyfrdyUkp31NVo934SJ0QLkX1-oeIxUrcDSiNLUGW7')
        personality_insights = PersonalityInsightsV3(
            version='2017-10-13',
            authenticator=authenticator
        )
        personality_insights.set_service_url('https://api.eu-gb.personality-insights.watson.cloud.ibm.com/instances/2d350496-9675-4cc6-8ecc-1b2919b69e5c')
        
        
        try:
            print("Hello")
        except ApiException as ex:
            print ("Method failed with status code " + str(ex.code) + ": " + ex.message)
        
        print("...fetching personality...\n")
        
        #Call to personality insight        
        authenticator = IAMAuthenticator('4_vyfrdyUkp31NVo934SJ0QLkX1-oeIxUrcDSiNLUGW7')
        personality_insights = PersonalityInsightsV3(
            version='2017-10-13',
            authenticator=authenticator
        )
        
        personality_insights.set_service_url('https://api.eu-gb.personality-insights.watson.cloud.ibm.com/instances/2d350496-9675-4cc6-8ecc-1b2919b69e5c')
        
        
        with open(join('', './profile_demo.json')) as profile_json:
            profile = personality_insights.profile(
                profile_json.read(),
                'application/json',
                content_type='application/json',
                consumption_preferences=True,
                raw_scores=True
            ).get_result()
        print(json.dumps(profile, indent=2))
        
        print ("\n\n\n________________________________\n\n\n")
        
        
print("Execution over")


Suspected: JamesJgm26
Hello
...fetching personality...

{
  "word_count": 215,
  "word_count_message": "There were 215 words in the input. We need a minimum of 600, preferably 1,200 or more, to compute statistically significant estimates",
  "processed_language": "en",
  "personality": [
    {
      "trait_id": "big5_openness",
      "name": "Openness",
      "category": "personality",
      "percentile": 0.45305429875000264,
      "raw_score": 0.7448860523765556,
      "significant": true,
      "children": [
        {
          "trait_id": "facet_adventurousness",
          "name": "Adventurousness",
          "category": "personality",
          "percentile": 0.35585581502582786,
          "raw_score": 0.4916738582785504,
          "significant": true
        },
        {
          "trait_id": "facet_artistic_interests",
          "name": "Artistic interests",
          "category": "personality",
          "percentile": 0.40557821037100505,
          "raw_score": 0.6532290775231826,